Processing Data

In [18]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import sys
cur = os.getcwd()
parent = os.path.abspath(os.path.join(cur,os.pardir))
sys.path.insert(0,parent)
import focus
%matplotlib inline

connection = sqlite3.connect("focus.db")
df = pd.read_sql("SELECT * FROM focus_logs",connection)
#The first thing is to add a tracking ended entry. So...
df

,id,date,time,focused,program,session_end
0,1,09/10/2024,13:13:44,● processing.ipynb - TimeLens - Visual Studio ...,Code.exe,0
1,2,09/10/2024,13:13:47,Outlook - Google Chrome,chrome.exe,0
2,3,09/10/2024,13:13:48,Mail - Kevin.Dsouza@cix.csi.cuny.edu - Outlook...,chrome.exe,0
3,4,09/10/2024,13:14:01,#vaping-corner | The Jade Palace - Discord,Discord.exe,0
4,5,09/10/2024,13:14:02,"Inbox (24,157) - kevindsouza476@gmail.com - Gm...",chrome.exe,0
...,...,...,...,...,...,...
228,229,09/10/2024,13:59:32,Task Switching,explorer.exe,0
229,230,09/10/2024,13:59:33,● processing.ipynb - TimeLens - Visual Studio ...,Code.exe,0
230,231,09/10/2024,13:59:34,Session End,None,1
231,232,09/10/2024,14:36:49,● processing.ipynb - TimeLens - Visual Studio ...,Code.exe,0


In [19]:
programInsights = pd.read_sql("SELECT * FROM program_insights",connection)
#The first thing is to add a tracking ended entry. So...
programInsights
#average_time standarization needed. reduce decimal points

,id,program,total_time,average_time,context_switch,last_updated
0,1,Code.exe,0 days 00:00:20,0 days 00:00:02.500000,8,2024-09-10 18:36:52
1,2,chrome.exe,0 days 00:15:21,0 days 00:00:34.111111111,27,2024-09-10 18:36:52
2,3,Discord.exe,0 days 00:01:05,0 days 00:00:09.285714285,7,2024-09-10 18:36:52
3,4,None,0 days 00:39:34,0 days 00:09:53.500000,4,2024-09-10 18:36:52
4,9,SearchApp.exe,0 days 00:00:10,0 days 00:00:02,5,2024-09-10 18:36:52
5,10,explorer.exe,0 days 00:00:27,0 days 00:00:01.928571428,14,2024-09-10 18:36:52
6,11,ONENOTE.EXE,0 days 00:25:09,0 days 00:01:00.360000,25,2024-09-10 18:36:52
7,12,GoogleDriveFS.exe,0 days 00:00:38,0 days 00:00:12.666666666,3,2024-09-10 18:36:52
8,21,ScreenClippingHost.exe,0 days 00:00:29,0 days 00:00:02.900000,10,2024-09-10 18:36:52


In [20]:
generalInsights = pd.read_sql("SELECT * FROM general_insights",connection)
#The first thing is to add a tracking ended entry. So...
generalInsights

,id,total_context_switches,last_updated
